In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet

In [2]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        self.classes = self.data['hotel_id'].unique().tolist()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.data.iloc[idx, 1]), str(self.data.iloc[idx, 0]))
        image = Image.open(img_name).convert("RGB")
        label = self.classes.index(self.data.iloc[idx, 1])
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# best_checkpoint_path = os.path.join("checkpoints", "efficientnet-b0_best_checkpoint.pth")

# model.load_state_dict(torch.load(best_checkpoint_path))

# model.eval()

# val_dataset = CustomDataset('validation.csv', 'final/validation_images', transform=data_transforms)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# correct = 0
# total = 0

# # Disable gradient calculation for evaluation
# with torch.no_grad():
#     for images, labels in val_loader:
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# accuracy = 100 * correct / total
# print(f"Best Model Validation Accuracy: {accuracy:.2f}%")


In [6]:
train_dataset = CustomDataset('train.csv', 'final/train_images', transform=data_transforms)

best_checkpoint_path = os.path.join("checkpoints_next20", "efficientnet-b0_best_checkpoint.pth")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

best_model = EfficientNet.from_pretrained('efficientnet-b0')
num_features = best_model._fc.in_features 
best_model._fc = nn.Linear(num_features, len(train_dataset.classes))
# best_model.load_state_dict(torch.load(best_checkpoint_path))
best_model.to(device)

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [7]:
# Load the state_dict, matching the keys correctly
# checkpoint = torch.load(best_checkpoint_path)
# state_dict = checkpoint['model_state_dict']

# # Update the state_dict keys to match the model's current state_dict
# state_dict = {k.replace("_fc.0", "_fc.0.weight").replace("_fc.2", "_fc.0.bias"): v for k, v in state_dict.items()}

# # Load the modified state_dict to the model
# best_model.load_state_dict(state_dict)

KeyError: 'model_state_dict'

In [9]:
state_dict = torch.load(best_checkpoint_path)
remove_prefix = 'module.'
state_dict = {k[len(remove_prefix):] if k.startswith(remove_prefix) else k: v for k, v in state_dict.items()}

# # Update the state_dict keys to match the model's current state_dict
state_dict = {k.replace("_fc.0.weight","_fc.weight").replace("_fc.0.bias","_fc.bias"): v for k, v in state_dict.items()}

# # Load the modified state_dict to the model
best_model.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
# Evaluate on the test dataset
val_dataset = CustomDataset('validation.csv', 'final/validation_images', transform=data_transforms)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

best_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = 100 * correct / total
print(f"Validation Accuracy: {val_accuracy:.2f}%")

Validation Accuracy: 59.72%
